# Preprocessing

In [1]:
import pandas as pd
import re
from collections import defaultdict

# ----------------------------
# 1. Chargement et nettoyage
# ----------------------------

df = pd.read_csv('call_center_final.csv')

def clean_text(text):
    if pd.isna(text):
        return "" #Vérifie si le texte est vide (NaN)
    text = text.lower() #Met tout en minuscules
    text = re.sub(r"[^a-zàâäéèêëîïôöùûüç0-9\s]", "", text) # Supprime les caractères spéciaux sauf :lettres (accentuées incluses), chiffres, espaces
    replacements = {
        r"\bpr\b": "pour",
        r"\bsvp\b": "s'il vous plaît",
        r"\bjedis\b": "je dis",
        r"\brathér\b": "plutôt",
        r"\bdactions\b": "d'actions"
    } # Remplace des abréviations ou mots fréquents
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return re.sub(r"\s+", " ", text).strip()

df['client_question_clean'] = df['client_question'].apply(clean_text)
df['agent_response_clean'] = df['agent_response'].apply(clean_text)

# ----------------------------
# 2. Extraction des entités
# ----------------------------
# Elle sert à extraire des entités spécifiques (comme un RIB, numéro de compte ou nom/prénom) depuis une colonne d’un DataFrame.
def extract_entities(row):
    entities = []
    text = row.get('client_responses', '')

    if pd.isna(text):
        return entities

    for rib in re.findall(r"\b\d{20}\b", text):
        start = text.find(rib)
        entities.append({
            "value": rib,
            "entity": "rib",
            "start": start,
            "end": start + len(rib)
        })

    for compte in re.findall(r"\b\d{11}\b", text):
        start = text.find(compte)
        entities.append({
            "value": compte,
            "entity": "num_compte",
            "start": start,
            "end": start + len(compte)
        })

    name_match = re.search(r"\b([A-Z]+),\s*([A-Z]+)\b", text)
    if name_match:
        nom = name_match.group(1)
        prenom = name_match.group(2)
        entities.append({
            "value": nom,
            "entity": "nom_client",
            "start": name_match.start(1),
            "end": name_match.end(1)
        })
        entities.append({
            "value": prenom,
            "entity": "prenom_client",
            "start": name_match.start(2),
            "end": name_match.end(2)
        })

    return entities

df['entities'] = df.apply(extract_entities, axis=1)

# ----------------------------
# 3. Mapping des intentions
# ----------------------------
# chaque intention exprimée naturellement est "mappée" (associée) à un code technique.

intent_mapping = {
    'Consultation du solde': 'consult_solde',
    'Demande d\'extrait de compte': 'demande_extrait',
    'Consultation des dernières opérations': 'consult_operations',
    'Consultation du nombre d’actions': 'consult_actions',
    'Autre question bancaire': 'autre_question'
}
df['intent_mapped'] = df['motif'].map(intent_mapping).fillna(df['motif'])
df = df.drop_duplicates(subset=['client_question_clean', 'intent_mapped'])

# ----------------------------
# 4. Annotation des entités dans le texte
# ----------------------------
# Elle sert à annoter (marquer) dans un texte les entités détectées, en les entourant avec une notation spéciale

def annotate_entities(text, entities):
    if pd.isna(text) or not entities:
        return text
    annotated = text
    for e in sorted(entities, key=lambda x: x['start'], reverse=True):
        start, end = e['start'], e['end']
        value = text[start:end]
        annotated = f"{annotated[:start]}[{value}]({e['entity']}){annotated[end:]}"
    return annotated

# ----------------------------
# 5. Génération du fichier Rasa nlu.yml
# ----------------------------

def generate_rasa_nlu_file(df):
    rasa_nlu_data = defaultdict(list)

    for _, row in df.iterrows():
        intent = row['intent_mapped']
        question = row['client_question_clean']
        entities = row.get('entities', [])
        annotated_text = annotate_entities(question, entities)
        if pd.notna(annotated_text):
            rasa_nlu_data[intent].append(annotated_text)

    with open("nlu.yml", "w", encoding="utf-8") as f:
        f.write("version: \"3.1\"\n")
        f.write("nlu:\n")
        for intent, examples in rasa_nlu_data.items():
            f.write(f"- intent: {intent}\n")
            f.write("  examples: |\n")
            for ex in examples:
                f.write(f"    - {ex}\n")

    print(" Fichier 'nlu.yml' généré avec succès.")

# ----------------------------
# 6. Exécution
# ----------------------------

generate_rasa_nlu_file(df)


Fichiers 'nlu_intents.yml' et 'nlu_entities.yml' générés avec succès.


In [9]:
import yaml

# Chemin vers le fichier nlu.yml
file_path = "nlu2.yml"

# Chargement du fichier YAML
with open(file_path, "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)

# Dictionnaire pour stocker le nombre d'exemples par intention
intent_counts = {}

# Parcourir les données
for nlu_entry in data.get("nlu", []):
    intent = nlu_entry.get("intent")
    examples = nlu_entry.get("examples", "")
    # Compter le nombre de lignes non vides après avoir enlevé le préfixe '-'
    if intent and examples:
        example_lines = [line.strip() for line in examples.strip().split('\n') if line.strip().startswith("-")]
        intent_counts[intent] = len(example_lines)

# Affichage des résultats
for intent, count in intent_counts.items():
    print(f"Intention '{intent}': {count} exemples")


Intention 'salutation': 26 exemples
Intention 'consult_solde': 169 exemples
Intention 'consult_actions': 109 exemples
Intention 'demande_extrait': 96 exemples
Intention 'consult_operations': 108 exemples
Intention 'autre_question': 208 exemples
Intention 'fournir_infos': 643 exemples
Intention 'goodbye': 73 exemples


# Split train/test nlu2.yml

In [3]:
import yaml
import random
from pathlib import Path

# Charger le fichier nlu.yml
with open("nlu2.yml", 'r', encoding='utf-8') as file:
    data = yaml.safe_load(file)

train_data = {"version": data.get("version", "3.1"), "nlu": []}
test_data = {"version": data.get("version", "3.1"), "nlu": []}

# Pour chaque intent, on divise les exemples
for intent_block in data['nlu']:
    intent_name = intent_block['intent']
    examples = intent_block['examples'].strip().split('\n')
    examples = [ex.strip() for ex in examples if ex.strip()]

    # Mélanger les exemples
    random.shuffle(examples)

    # Split 70% train, 30% test
    split_idx = int(0.7 * len(examples))
    train_examples = examples[:split_idx]
    test_examples = examples[split_idx:]

    # Ajouter au fichier train
    if train_examples:
        train_data['nlu'].append({
            'intent': intent_name,
            'examples': '\n'.join(train_examples)
        })

    # Ajouter au fichier test
    if test_examples:
        test_data['nlu'].append({
            'intent': intent_name,
            'examples': '\n'.join(test_examples)
        })

# Sauvegarder les fichiers
with open("nlu2_train.yml", 'w', encoding='utf-8') as train_file:
    yaml.dump(train_data, train_file, allow_unicode=True, sort_keys=False)

with open("nlu2_test.yml", 'w', encoding='utf-8') as test_file:
    yaml.dump(test_data, test_file, allow_unicode=True, sort_keys=False)

print("Fichiers nlu22_train.yml et nlu22_test.yml générés avec succès.")


Fichiers nlu2_train.yml et nlu2_test.yml générés avec succès.
